# Library

In [188]:
import sys
sys.path.append('../../')
import PreProcessingUtils as ppu
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

# Read Files

In [189]:
df_boards = pd.read_csv('../../CleanedCSV/boards_cleaned.csv')
df_members = pd.read_csv('../../CleanedCSV/members_cleaned.csv')
df_posts = pd.read_csv('../../CleanedCSV/posts_cleaned.csv')
df_threads = pd.read_csv('../../CleanedCSV/threads_cleaned.csv')

In [190]:
df_boards.shape, df_members.shape, df_posts.shape, df_threads.shape

((446, 2), (52406, 5), (294596, 7), (75122, 6))

In [191]:
df_boards_threads = pd.merge(df_boards, df_threads, left_on='id', right_on='board_id', suffixes=('_board', '_thread')).drop(columns='board_id')
print(df_boards_threads.shape)
df_boards_threads.head()

(75122, 7)


,id_board,name_board,id_thread,creator,creator_id,name_thread,created_on
0,212,Tor,1713741075,blonger,32544,"Tor shrinked of about 13,5% but you can contri...",2020-01-09
1,212,Tor,2058062672,Xanitforthecash,2467,should I run a tor node,2020-01-09
2,212,Tor,2056740678,Syndicate,33036,Tor Browser 9.0!,2019-11-06
3,212,Tor,1210687208,Syndicate,33036,[UPDATED] Tor Security Guide Crosspost,2019-11-06
4,212,Tor,1655155960,wekhiu48,1178,"/u/CostcoRotisserieChicken spreading FUD, Let'...",2020-01-09


In [192]:
df_boards_threads_members = pd.merge(df_boards_threads, df_members, left_on=['creator_id', 'creator'], right_on=['id', 'username'], suffixes=('_thread', '_member')).drop(columns=['id', 'creator'])
print(df_boards_threads_members.shape)
df_boards_threads_members.head()

(75122, 10)


,id_board,name_board,id_thread,creator_id,name_thread,created_on,username,registration_date,total_posts,reputation
0,212,Tor,1713741075,32544,"Tor shrinked of about 13,5% but you can contri...",2020-01-09,blonger,1900-01-01 00:00:00,0.0,0.0
1,212,Tor,2058062672,2467,should I run a tor node,2020-01-09,Xanitforthecash,2018-03-21 00:00:00,88.0,9.0
2,212,Tor,2056740678,33036,Tor Browser 9.0!,2019-11-06,Syndicate,1900-01-01 00:00:00,0.0,0.0
3,212,Tor,1210687208,33036,[UPDATED] Tor Security Guide Crosspost,2019-11-06,Syndicate,1900-01-01 00:00:00,0.0,0.0
4,212,Tor,1655155960,1178,"/u/CostcoRotisserieChicken spreading FUD, Let'...",2020-01-09,wekhiu48,2019-05-24 00:00:00,1355.0,342.0


In [193]:
df_final = pd.merge(df_boards_threads_members, df_posts, left_on=['id_thread', 'id_board'], right_on=['thread_id', 'board_id'], suffixes=('', '_post')).drop(columns=['id', 'id_thread', 'id_board', 'thread_id', 'board_id'])
print(df_final.shape)
df_final.head()

(294596, 12)


,name_board,creator_id,name_thread,created_on,username,registration_date,total_posts,reputation,creator,creator_id_post,content,created_on_post
0,Tor,32544,"Tor shrinked of about 13,5% but you can contri...",2020-01-09,blonger,1900-01-01 00:00:00,0.0,0.0,[deleted],37,[removed],2019-10-16
1,Tor,32544,"Tor shrinked of about 13,5% but you can contri...",2020-01-09,blonger,1900-01-01 00:00:00,0.0,0.0,blonger,32544,The Torproject recently ***LINK***announced[ht...,2019-10-30
2,Tor,2467,should I run a tor node,2020-01-09,Xanitforthecash,2018-03-21 00:00:00,88.0,9.0,rswz,32661,***LINK***http://ea5faa5po25cf7fb.onion/projec...,2019-10-16
3,Tor,2467,should I run a tor node,2020-01-09,Xanitforthecash,2018-03-21 00:00:00,88.0,9.0,ChemistWeb,33150,if someone will do something bad from that ip ...,2019-10-16
4,Tor,2467,should I run a tor node,2020-01-09,Xanitforthecash,2018-03-21 00:00:00,88.0,9.0,Xanitforthecash,2467,I already leave tor open for a good amount of ...,2019-10-16


# Preprocess Df-Final

In [194]:
df_final.drop_duplicates(inplace=True)
df_final.dropna(subset='content', inplace=True)
print(df_final.shape)
df_final.head()

(292783, 12)


,name_board,creator_id,name_thread,created_on,username,registration_date,total_posts,reputation,creator,creator_id_post,content,created_on_post
0,Tor,32544,"Tor shrinked of about 13,5% but you can contri...",2020-01-09,blonger,1900-01-01 00:00:00,0.0,0.0,[deleted],37,[removed],2019-10-16
1,Tor,32544,"Tor shrinked of about 13,5% but you can contri...",2020-01-09,blonger,1900-01-01 00:00:00,0.0,0.0,blonger,32544,The Torproject recently ***LINK***announced[ht...,2019-10-30
2,Tor,2467,should I run a tor node,2020-01-09,Xanitforthecash,2018-03-21 00:00:00,88.0,9.0,rswz,32661,***LINK***http://ea5faa5po25cf7fb.onion/projec...,2019-10-16
3,Tor,2467,should I run a tor node,2020-01-09,Xanitforthecash,2018-03-21 00:00:00,88.0,9.0,ChemistWeb,33150,if someone will do something bad from that ip ...,2019-10-16
4,Tor,2467,should I run a tor node,2020-01-09,Xanitforthecash,2018-03-21 00:00:00,88.0,9.0,Xanitforthecash,2467,I already leave tor open for a good amount of ...,2019-10-16


In [195]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 292783 entries, 0 to 294595
Data columns (total 12 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   name_board         292783 non-null  object 
 1   creator_id         292783 non-null  int64  
 2   name_thread        292783 non-null  object 
 3   created_on         292783 non-null  object 
 4   username           291854 non-null  object 
 5   registration_date  283901 non-null  object 
 6   total_posts        283901 non-null  float64
 7   reputation         283901 non-null  float64
 8   creator            292783 non-null  object 
 9   creator_id_post    292783 non-null  int64  
 10  content            292783 non-null  object 
 11  created_on_post    292783 non-null  object 
dtypes: float64(2), int64(2), object(8)
memory usage: 29.0+ MB


In [196]:
df_final.to_csv('merged_data.csv', index=False)